# Data Preprocessing

calendar.csv: contains information such as dates, holidays, events, etc

sales_train_validation.csv: daily sales of each product, rows are products, columns are d_1, d_2...

sell_prices.csv: the selling price of a product in a store in a certain week

sample_submission.csv: format for submitting predictions

sales_train_evaluation.csv: contains some evaluation data

In [3]:
import pandas as pd
import holidays

In [4]:
# Load datasets
calendar_df = pd.read_csv('Dataset/calendar.csv')
sales_val_df = pd.read_csv('Dataset/sales_train_validation.csv')
sell_prices_df = pd.read_csv('Dataset/sell_prices.csv')

In [5]:
# Convert sales data from wide to long format
sales_long_df = pd.melt(sales_val_df,
                        id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
                        var_name='d',
                        value_name='sales')

# Merge sales data with calendar
sales_calendar_df = sales_long_df.merge(calendar_df, on='d', how='left')

# Merge with sell prices
full_df = sales_calendar_df.merge(sell_prices_df,
                                  on=['store_id', 'item_id', 'wm_yr_wk'],
                                  how='left')

In [6]:
# Create basic date-related features
full_df['date'] = pd.to_datetime(full_df['date'])
full_df['day_of_week'] = full_df['date'].dt.dayofweek
full_df['month'] = full_df['date'].dt.month
full_df['year'] = full_df['date'].dt.year
full_df['is_weekend'] = full_df['day_of_week'].isin([5, 6]).astype(int)

In [7]:
# Add US holiday features
us_holidays = holidays.US()
full_df['is_us_holiday'] = full_df['date'].isin(us_holidays).astype(int)
full_df['us_holiday_name'] = full_df['date'].map(us_holidays)

In [8]:
# Sort by date for lag features
full_df = full_df.sort_values(['store_id', 'item_id', 'date'])

# Generate lag features (e.g., lag 7 days)
full_df['sales_lag_7'] = full_df.groupby(['store_id', 'item_id'])['sales'].shift(7)

# Rolling mean features (past 7-day sales average)
full_df['rolling_mean_7'] = full_df.groupby(['store_id', 'item_id'])['sales'].transform(lambda x: x.shift(1).rolling(window=7).mean())

# Price-related features
full_df['price_change'] = full_df.groupby(['store_id', 'item_id'])['sell_price'].pct_change()

# Optimize memory usage
for col in ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']:
    full_df[col] = full_df[col].astype('category')

/var/folders/zv/n9dl8wj94hqfxs9swp91wt0w0000gn/T/ipykernel_83024/287304803.py:11: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  full_df['price_change'] = full_df.groupby(['store_id', 'item_id'])['sell_price'].pct_change()


In [11]:
# Save processed data for modeling
full_df.to_pickle('Dataset/processed_sales_data.pkl')

In [13]:
print(full_df.head())

                                 id      item_id  dept_id cat_id store_id  \
1612    FOODS_1_001_CA_1_validation  FOODS_1_001  FOODS_1  FOODS     CA_1   
32102   FOODS_1_001_CA_1_validation  FOODS_1_001  FOODS_1  FOODS     CA_1   
62592   FOODS_1_001_CA_1_validation  FOODS_1_001  FOODS_1  FOODS     CA_1   
93082   FOODS_1_001_CA_1_validation  FOODS_1_001  FOODS_1  FOODS     CA_1   
123572  FOODS_1_001_CA_1_validation  FOODS_1_001  FOODS_1  FOODS     CA_1   

       state_id    d  sales       date  wm_yr_wk  ... snap_TX  snap_WI  \
1612         CA  d_1      3 2011-01-29     11101  ...       0        0   
32102        CA  d_2      0 2011-01-30     11101  ...       0        0   
62592        CA  d_3      0 2011-01-31     11101  ...       0        0   
93082        CA  d_4      1 2011-02-01     11101  ...       1        0   
123572       CA  d_5      4 2011-02-02     11101  ...       0        1   

        sell_price  day_of_week is_weekend is_us_holiday us_holiday_name  \
1612           2